In [2]:
from PIL import Image
import os
import csv
import exifread
from PIL.ExifTags import TAGS
from exif import Image

In [20]:
master_directory = "C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\independent_imgs"
csv_file = "mxw_agl_gsd.csv"

csv_header = ["image_filename", "transect_id", "altitude", "gsd"]

with open(csv_file, mode='w', newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    csv_writer.writerow(csv_header)

In [6]:
# Open an example image and extract EXIF data for altitude (reference for getting any EXIF tag)
image_path = "C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\independent_imgs\\9\\EP-01-26106_0010_0488.JPG"

with open(image_path, 'rb') as src:
    img = Image(src)
    #print(img.list_all()) # this is where you can get all the EXIF tag names to get relevant values
    altitude = img.get("gps_altitude")
    print(altitude)

1900.4636


In [21]:
# Get altitude and GSD for all images

ground_elevation = 1844

# Define sensor parameters (in millimeters)
image_width = 5472  # Example image width in pixels
image_height = 3648  # Example image height in pixels
sensor_width = 0.0132  # Make sure this is converted to meters!
sensor_height = 0.0088  # Make sure this is converted to meters!
focal_length = 0.0106  # Make sure this is converted to meters!

altitude_data = []  # List to store extracted altitude values

for transect_id in os.listdir(master_directory):
    transect_dir = os.path.join(master_directory, transect_id)
    
    if os.path.isdir(transect_dir):
        for image_filename in os.listdir(transect_dir):
            if image_filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(transect_dir, image_filename)
                    
                # Open the image and extract EXIF data
                with open(image_path, 'rb') as src:
                    img = Image(src)
    
                    altitude = img.get("gps_altitude")
                # Calculate altitude above ground level
                agl_altitude = altitude - ground_elevation if altitude is not None else None
                altitude_data.append((image_filename, transect_id, agl_altitude))

# Calculate GSD and write data to CSV
with open(csv_file, mode='a', newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    
    for image_filename, transect_id, agl_altitude in altitude_data:
        if agl_altitude is not None:
            # Calculate ground sampling distance (in meters/px)
            gsd_m = (sensor_width * agl_altitude) / (focal_length * image_width)
            gsd = gsd_m*100 #convert to cm-- optional
        else:
            gsd = None

        # Write data to CSV
        csv_writer.writerow([image_filename, transect_id, agl_altitude, gsd])
